In [146]:
import pandas as pd
import numpy as np
import json
import lightgbm as lgb

In [266]:
def rename_columns(data):
    trafo_dict = {x:int(y) for x, y in zip( data.columns, np.linspace(0, data.shape[1]-1, data.shape[1]))}
    data = data.rename(columns = trafo_dict)
    return data

def read_data():
    df_train = pd.read_table("./train/train.txt", sep = ",", header = None)
    df_test = pd.read_table("./test/apply_new.txt", sep = ",", header = None)
    df_train.insert(df_train.shape[1], df_train.shape[1], df_train.iloc[:, 1])
    df_train.drop(1, axis = 1, inplace = True)
    df_train = rename_columns(df_train)
    test_user_id = df_test.iloc[:, 0]
    return df_train, df_test, test_user_id

def feature_engineering(data):
    data = data.fillna(-1)  
    data.iloc[:, 3] = data.iloc[:, 3].apply(json.loads)
    data.iloc[:, 3] = data.iloc[:, 3].apply(lambda x: x[0])
    data.drop(4, axis = 1, inplace=True)
    data = rename_columns(data)
    data.iloc[:, :] = data.iloc[:, :].astype("category")
    return data

def get_train_val_test_data(df_train, df_test, ratio):
    All_X_train = df_train.iloc[:, 1:-1]
    X_train = df_train.iloc[:int(df_train.shape[0] * ratio), 1:-1]
    X_val = df_train.iloc[int(df_train.shape[0] * ratio): , 1:-1]
    X_test = df_test.iloc[: , 1:]

    All_y_train = df_train.iloc[:, -1]
    y_train = df_train.iloc[:int(df_train.shape[0] * ratio), -1]
    y_val = df_train.iloc[int(df_train.shape[0] * ratio):, -1]
    return All_X_train, All_y_train, X_train, X_val, X_test, y_train, y_val

In [267]:
df_train, df_test, test_user_id = read_data()
df_train, df_test = feature_engineering(df_train), feature_engineering(df_test)
All_X_train, All_y_train, X_train, X_val, X_test, y_train, y_val = get_train_val_test_data(df_train, df_test, 0.7)

In [221]:
X_train

,1,2,3,4,5,6,7
0,-1.0,-1.0,4457057,广西,北海,华为,华为 mate20pro
1,-1.0,5.0,10577375,广东,广州,OPPO,r11
2,-1.0,-1.0,11171956,内蒙古,锡林郭勒盟,小米,小米 红米note2
3,-1.0,2.0,4457927,四川,成都,vivo,vivo x20
4,-1.0,-1.0,5737867,湖南,长沙,vivo,x23
...,...,...,...,...,...,...,...
209995,-1.0,3.0,13669217,江苏,宿迁,小米,小米 6
209996,0.0,5.0,13320587,北京,北京,荣耀,v8
209997,-1.0,3.0,10573044,湖北,襄阳,华为,华为 畅享9
209998,-1.0,1.0,10158594,湖南,常德,OPPO,r9s


In [222]:
X_val

,1,2,3,4,5,6,7
210000,-1.0,3.0,11180532,河南,平顶山,诺基亚,x6
210001,-1.0,3.0,12545416,陕西,安康,华为,华为 畅想9plus
210002,-1.0,4.0,12805457,江苏,淮安,小米,小米 6x
210003,-1.0,-1.0,5674849,河北,唐山,小米,小米 6
210004,-1.0,4.0,12546375,广东,云浮,OPPO,r9
...,...,...,...,...,...,...,...
299995,1.0,3.0,12545416,陕西,西安,OPPO,r9
299996,-1.0,-1.0,10882324,河南,商丘,OPPO,OPPO r11
299997,-1.0,2.0,9302361,福建,厦门,OPPO,r9s
299998,-1.0,6.0,10560059,山西,太原,OPPO,a7x


In [223]:
len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test),  

(210000, 210000, 90000, 90000, 100000, 100000)

In [224]:
X_test

,1,2,3,4,5,6,7
0,-1.0,5.0,4447574,北京,北京,华为,nova2
1,-1.0,3.0,11185398,江苏,连云港,魅族,魅族 note8
2,0.0,1.0,13498871,四川,德阳,OPPO,a57
3,-1.0,3.0,12123770,辽宁,营口,vivo,vivo y75
4,-1.0,-1.0,12518145,广西,南宁,vivo,vivo y97
...,...,...,...,...,...,...,...
99995,-1.0,1.0,91325,山东,临沂,OPPO,r11splus
99996,-1.0,3.0,11871458,安徽,池州,OPPO,a5
99997,-1.0,-1.0,10567612,山东,菏泽,vivo,vivo y66i
99998,-1.0,-1.0,10757291,四川,雅安,vivo,vivo x20


In [184]:
list(X_train.columns)

[2, 3, 4, 5, 6, 7, 8]

In [281]:
print("starting first testing......")  
clf = lgb.LGBMClassifier(  
        boosting_type='gbdt', num_leaves=50, reg_alpha=0.0, reg_lambda=1,  
        max_depth=-1, n_estimators=1500, objective='binary',  
        subsample=1, colsample_bytree=1, subsample_freq=1,  
        learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=100  
    )  

clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='auc',early_stopping_rounds=200, categorical_feature=list(X_train.columns))  

starting first testing......
/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [1, 2, 3, 4, 5, 6, 7]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[1]	valid_0's auc: 0.713335	valid_0's binary_logloss: 0.685696
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.71544	valid_0's binary_logloss: 0

LGBMClassifier(colsample_bytree=1, learning_rate=0.05, min_child_weight=50,
               n_estimators=1500, n_jobs=100, num_leaves=50, objective='binary',
               random_state=2018, reg_lambda=1, subsample=1, subsample_freq=1)

In [278]:
clf.best_iteration_

78

In [283]:
clf_all = lgb.LGBMClassifier(  
        boosting_type='gbdt', num_leaves=50, reg_alpha=0.0, reg_lambda=1,  
        max_depth=-1, n_estimators=clf.best_iteration_+10, objective='binary',  
        subsample=1, colsample_bytree=1, subsample_freq=1,  
        learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=100,
        verbose=1
    )  

clf_all.fit(All_X_train, All_y_train, categorical_feature=list(X_train.columns))  

/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[LightGBM] [Info] Number of positive: 150000, number of negative: 150000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5508
[LightGBM] [Info] Number of data points in the train set: 300000, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


LGBMClassifier(colsample_bytree=1, learning_rate=0.05, min_child_weight=50,
               n_estimators=88, n_jobs=100, num_leaves=50, objective='binary',
               random_state=2018, reg_lambda=1, subsample=1, subsample_freq=1,
               verbose=1)

In [286]:
import sklearn
y_val_pre = clf.predict(X_val)  
print(sklearn.metrics.f1_score(y_val, y_val_pre, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn'))
y_val_pre = clf_all.predict(X_val)  
print(sklearn.metrics.f1_score(y_val, y_val_pre, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn'))

0.6631535464278936
0.6975382182980193


In [287]:
y_test_pre = clf_all.predict(X_test)
y_test_pre  

array([0, 0, 0, ..., 1, 0, 0])

In [288]:
import time
t = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
res = pd.DataFrame(index = test_user_id, data = y_test_pre)
res.index.name = "user_id"
res.to_csv(r"./sub/sub_{}.csv".format(t), header=["category_id"])